In this notebook, we are going to see how a constraint can be activated under a variable-based condition (ie, an other constraint).
The small problem we are going to solve is the following:

%%latex
\begin{equation*}
  f(a,b)=\begin{cases}
    a\times b, & \text{if $a>b$}.\\
    a+b, & \text{otherwise}.
  \end{cases}
\end{equation*}

There are three ways to handle such conditional constraint in Choco.

In [3]:
%%loadFromPOM
<repository>
  <id>oss-sonatype-snapshots</id>
  <url>https://oss.sonatype.org/content/repositories/snapshots/</url>
</repository>
<dependency>
  <groupId>org.choco-solver</groupId>
  <artifactId>choco-solver</artifactId>
  <version>4.10.0</version>
</dependency>

In [9]:
import org.chocosolver.solver.Model;
import org.chocosolver.solver.variables.IntVar;
import org.chocosolver.solver.Solver;

## First approach
In the first approach, we are going to use a `ifThenElse` constraint that takes three constraints as parameters.
The first constraint is the conditional, the second is the one that is satisfied when the condition holds, 
the third is the one that is satified when the condition doesn't hold. 

In [16]:
{
    Model model = new Model("1st try");
    IntVar A = model.intVar("A", 0, 3);
    IntVar B = model.intVar("B", 0, 3);
    IntVar F = model.intVar("F", 0, 10);
    
    model.ifThenElse(
        model.arithm(A,">",B), // if A > B
        model.times(A, B, F), // then F = A * B
        model.arithm(A,"+",B,"=",F)); // else F = A + B

    Solver solver = model.getSolver();
    solver.printShortFeatures();
    while(solver.solve()){
        System.out.printf("A=%d, B=%d, H=%d\n", A.getValue(), B.getValue(), F.getValue());
    }
    solver.printShortStatistics();
}

Model[1st try], 9 variables, 5 constraints, building time: 0,001s, w/o user-defined search strategy, w/o complementary search strategy
A=0, B=1, H=1
A=0, B=2, H=2
A=0, B=3, H=3
A=1, B=1, H=2
A=1, B=2, H=3
A=1, B=3, H=4
A=2, B=3, H=5
A=3, B=3, H=6
A=0, B=0, H=0
A=2, B=2, H=4
A=1, B=0, H=0
A=2, B=0, H=0
A=3, B=0, H=0
A=2, B=1, H=2
A=3, B=1, H=3
A=3, B=2, H=6
Model[1st try], 16 Solutions, Resolution time 0,013s, 35 Nodes (2 602,5 n/s), 39 Backtracks, 0 Backjumps, 4 Fails, 0 Restarts


In that approach, some additional variables and constraints are introduced to make sure the relation holds.
It relies on reification which attaches a boolean variable to a constraint, which is set to `true` when the constraint is satified, 
`false` otherwise (and vice versa). In this approach, the decomposition is automatically done by the model on the call to `ifThenElse`.

## Second approach: using expressions
Variables can be combined together to form an expression. An expression can be arithmetical, logical or relational.
Once declared, any expression can be turned into an integer variable (for arithmetical expression), a boolean variable (for logical expression) or a constraint (for relationnal one). 
The latter can either be posted as a combination of constraints or as a table constraint. In that case, the tuple are generated from the expression (this may not be "free" in term of time and space).

In [19]:
{
    Model model = new Model("1st try");
    IntVar A = model.intVar("A", 0, 3);
    IntVar B = model.intVar("B", 0, 3);
    IntVar F = model.intVar("F", 0, 10);
    
    F.eq( // F is equal to ...
        A.gt(B).ift( // if A > B 
            A.mul(B), // then A * B
            A.add(B)  // else A + B
        )
    ).decompose() // decompose the expression
    //).extension() // generate a table constraint
    .post(); // post the constraint

    Solver solver = model.getSolver();
    solver.printShortFeatures();
    while(solver.solve()){
        System.out.printf("A=%d, B=%d, F=%d\n", A.getValue(), B.getValue(), F.getValue());
    }
    solver.printShortStatistics();
}

Model[1st try], 3 variables, 1 constraints, building time: 0,004s, w/o user-defined search strategy, w/o complementary search strategy
A=0, B=0, F=0
A=1, B=0, F=0
A=2, B=0, F=0
A=3, B=0, F=0
A=0, B=1, F=1
A=1, B=1, F=2
A=2, B=1, F=2
A=3, B=1, F=3
A=0, B=2, F=2
A=1, B=2, F=3
A=2, B=2, F=4
A=3, B=2, F=6
A=0, B=3, F=3
A=1, B=3, F=4
A=2, B=3, F=5
A=3, B=3, F=6
Model[1st try], 16 Solutions, Resolution time 0,011s, 31 Nodes (2 794,8 n/s), 31 Backtracks, 0 Backjumps, 0 Fails, 0 Restarts


In this approach again, the model automatically decomposes the relation into constraints and introduces some variables. One may replace `.decompose()` by `.extension()` and note that no constraint or variable is added. 

## Third approach: DIY
In that approach, constraints are reified with boolean variables and those variables are, in turn, linked to an `if then` relation.

In [ ]:
{
    Model model = new Model();
    IntVar A = model.intVar("A", 0, 3);
    IntVar B = model.intVar("B", 0, 3);
    IntVar F = model.intVar("F", 0, 10);

    BoolVar b = model.arithm(A, ">", B).reify();
    model.times(A, B, F).reifyWith(b);
    model.arithm(A, "+", B, "=", F).reifyWith(b.not());

    Solver solver = model.getSolver();
    solver.printShortFeatures();
    while(solver.solve()){
        System.out.printf("A=%d, B=%d, H=%d\n", A.getValue(), B.getValue(), F.getValue());
    }
    solver.printShortStatistics();
}

The `A>B` constraint is reified with `b` a boolean variable which is then used to reify the second constraint.
Then, the refutation of `b` is used to reified the third constraint. 

When a co

## Using LNS
Large Neighborhood Search mimics a local search algorithm: it freezes some variables to their value in the last solution and let the others being fixed by the search strategy. 
Ususally, the user let the solver finds the first solution. But, it is also possible to load a solution to start from.

The first step is to load the solution into a `Solution` object.

In [ ]:
import org.chocosolver.solver.Solution;
import java.util.stream.IntStream;

Solution sol = new Solution(model, ticks);
IntStream.range(0, m).forEach(i -> sol.setIntVal(ticks[i], good[i]));

Then, a LNS can be declared:

In [ ]:
import org.chocosolver.solver.search.limits.FailCounter;
import org.chocosolver.solver.search.loop.lns.INeighborFactory;
import org.chocosolver.solver.search.loop.lns.neighbors.INeighbor;
import org.chocosolver.solver.search.loop.move.MoveLNS;


MoveLNS lns = new MoveLNS(
    model.getSolver().getMove(), // this represents the default way to explore the search, ie DFS 
    INeighborFactory.random(ticks), // this represents the way neighborhood will be generated, here randomly
    new FailCounter(model, 100) // this represents when to try a new neighborhood, here every 100 failures
);  
lns.loadFromSolution(sol, model.getSolver()); // the solution is then loaded
model.getSolver().setMove(lns); // an LNS is declared

Now, the search can start:

In [ ]:
model.getSolver().showSolutions(() -> Arrays.toString(ticks));
model.getSolver().limitTime("5s"); // to avoid never ending loop
model.getSolver().solve(); // look for the first improving solution

## Using LDS
Limited-Discrepancy Search bets that the enumeration strategy (variable selection + value selection) is quite enough.
So it tries not to move too much away from it (see "W.D. Harvey and M.L.Ginsberg, *Limited Discrepancy Search*, IJCAI-95" for more details).
This approach requires to declare an enumeration strategy based on the solution.

In [ ]:
import org.chocosolver.solver.search.strategy.Search;
import org.chocosolver.solver.search.strategy.selectors.variables.InputOrder;

import java.util.Arrays;
import java.util.Map;
import java.util.stream.Collectors;

model.getSolver().hardReset(); // required for notebook 

Map<IntVar, Integer> map = IntStream.range(0,m).boxed().collect(Collectors.toMap(i->ticks[i], i -> good[i]));
model.getSolver().setSearch(Search.intVarSearch(
        new InputOrder<>(model), // variable selection: from ticks[0] to ticks[m-1]
        var -> { // value selection, choose value from solution if possible
            if(var.contains(map.get(var))){
                return map.get(var);
            }
            return var.getLB(); // otherwise, select the current lower bound
        },
        ticks
));
model.getSolver().setLDS(12); // discrepancy is set to 12
model.getSolver().showSolutions(() -> Arrays.toString(ticks));
model.getSolver().limitTime("5s");
model.getSolver().solve();

## Solution-based phase saving approach
This approach is a kind of compromise between LNS and LDS. It requires a solution to be applied (most of the time, found by the solver) and then when a value selection is called, it tries first the value in the last solution found (for more details, see "E. Demirović, G. Chu and P.J. Stuckey, *Solution-based phase saving for CP: A value-selection heuristic to simulate local search behavior in complete solvers*, CP18").

In [ ]:
import org.chocosolver.solver.search.strategy.Search;
import org.chocosolver.solver.search.strategy.selectors.variables.InputOrder;
import org.chocosolver.solver.search.strategy.selectors.values.IntDomainLast;
import org.chocosolver.solver.search.strategy.selectors.values.IntDomainMin;

model.getSolver().hardReset(); // required for notebook

model.getSolver().setSearch(Search.intVarSearch(
    new InputOrder<>(model), // variable selection: from ticks[0] to ticks[m-1]
    new IntDomainLast(sol, new IntDomainMin()), // value selection, and default one if value from solution is forbidden
    ticks
));
model.getSolver().showSolutions(() -> Arrays.toString(ticks));
model.getSolver().limitTime("5s");
model.getSolver().solve();